In [ ]:
#runSim
import random
from simSetting import Model
from netTopology import Topology
from sensors import Sensors
from plotter import Plotter
import simCore as core
import brodcasting as bc
import setting
import packetlossHistory

#################### Network Parameters ####################
# number of patient
n = 10

# number of sensor for each patient
m = 5

# senesing area
x = 100; y=100

# set paramaters
model = Model(n,m,x,y)

# create topology
top = Topology(model)

# create sensors
numNodes = n+n*m
sensors = [Sensors() for _ in range(numNodes+1)]

# configuration nodes
for i in range(10,numNodes): #########changed#######
    # coordinator nodes
    if i<n:
        counterType = m    #Type of sensor (is coordinator)
        mch = numNodes     #Sensor is member of which coordinator(Sink)
        coordiantorType = i % 4
    # sensor nodes
    else:
        mch = (i // m) - 2     #Type of sensor (is one of 5 types)
        sensorType = i % m    #Sensor is member of which coordinator(coordinator)
        coordiantorType = None
    #print(i, mch)
    sensors[i].Configuration(top.x[i], top.y[i], i, mch, sensorType, coordiantorType)

# configuration sink
sensors[(numNodes)].Configuration(model.Sinkx, model.Sinky, numNodes, -1, -1)

# create neighbours list
setting.init()
packetlossHistory.init(model)

#################### Evaluation Parameters ####################
flag_first_dead=0                                  # flag_first_dead
first_dead=0                                       # round

initEnergy=0                                       # initial energy
for i in range(n):
    initEnergy=sensors[i].E+initEnergy

SRP = []                                           # number of sent routing packets
RRP = []                                           # number of receive routing packets
SDP = []                                           # number of sent data packets
RDP = []                                           # number of receive data packets
Delay = []
Sum_DEAD = []
ConsumptionEnergy = []

#################### Start Simulation ####################
# Sink broadcast the message of starting netwok....
sensors = bc.downwardBrodcasting(sensors, model, 'Hello')
core.init(model)
core.createNeighbourList(sensors, model)

for r in range(model.rmax):
    setting.init()
    delay = 0

    # Nodes movement
    top.RandomWalk(model)
    for i in range(numNodes):
        sensors[i].Moving(top.x[i], top.y[i])

    core.updateNeighbourList(sensors, model)
    core.updateCore(sensors, model) ##########!!!!!!!!##############

    # Gathering data in SDN
    sensors = bc.upwardBrodcasting(sensors, model, 'Hello')

    # Detection Routes by SDN
    core.updateRoutingTable(sensors, model)

    # Sending paths
    sensors = bc.downwardBrodcasting(sensors, model, 'Hello')

    # Sensing
    for node in sensors:
        node.Sensing(model)
        node.Proccessing(model)

    # Send Data to Coordinator
    for node in sensors:
        if (node.E > 0 and node.type == 'N'):
            core.sendPacketToCoordinator(sensors, model, [node.id], 'Data')
            #sensors[node.MCH].Proccessing(model)

    # Send data from coordinators to the sink
    for node in sensors:
        if (node.E > 0 and node.type == 'C'):
            packetloss = 0
            patientClass = node.patientClass

            for s in range(model.nPackets):

                r = random.random()
                if (r < 0.8):
                    dataPriority = 0   #'low'
                elif (.8 <= r < .95):
                    dataPriority = 1   #'medium'
                else:
                    dataPriority = 2   #'high'

                finallPriority = max([patientClass, dataPriority])

                if finallPriority == 0:
                    finallPriority = 'low'
                elif finallPriority == 1:
                    finallPriority = 'medium'
                else:
                    finallPriority = 'high'

                core.sendPacketToSink(sensors, model, [node.id], 'Data', finallPriority)




